In [1]:
PYTHON_PATHS = [".."]
import sys
for path in PYTHON_PATHS:
    if path not in sys.path:
        sys.path.append(path)

In [2]:
import numpy as np
import os
from tqdm.notebook import tqdm as tqdm
import matplotlib.pyplot as plt

In [3]:
from unsupkeypoints.data import DescriptorDataModule
from unsupkeypoints.models import PointNetwork
from unsupkeypoints.criterions import RegressionLoss, RGBandModelReprojectionLoss
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.utilities.parsing import AttributeDict
import torch
import pytorch_lightning as pl

In [4]:
from clearml import Task

In [5]:
task = Task.init(project_name="unsup-3d-keypoints", task_name="3d position from descriptor")

ClearML Task: created new task id=c4869443af6043d4819d0a8df3af7c86
ClearML results page: https://app.community.clear.ml/projects/f773f19fb1d94708840bc5460e97ba2b/experiments/c4869443af6043d4819d0a8df3af7c86/output/log


# Data module

In [7]:
from clearml import Dataset
dataset_path = Dataset.get(dataset_id="40300e50677746148777e4a3c3a21ba9").get_local_copy()

In [8]:
TRAIN_PATH = os.path.join(dataset_path, "output_kapture/7scenes/fire/mapping")
TEST_PATH = os.path.join(dataset_path, "output_kapture/7scenes/fire/query")

In [9]:
data_module = DescriptorDataModule(TRAIN_PATH, TEST_PATH)

[DescriptorDataModule] - train dataset size 568196
[DescriptorDataModule] - test dataset size 544913


# Model

In [10]:
params = AttributeDict(
    name="point_net",
    optimizer=AttributeDict(),
    hidden_dimensions=(100,),
    input_dimension=512,
)
task.connect(params)

{'name': 'point_net',
 'optimizer': {},
 'hidden_dimensions': (100,),
 'input_dimension': 512}

In [11]:
criterion_params = {
    "minimal_depth": 0.1,
    "maximal_distance": 0.2,
    "maximal_reprojection_loss": 1000,
    "robust_maximal_reprojection_loss": 100,
    "distance_coef": 500
}
task.connect(criterion_params)
criterion = RGBandModelReprojectionLoss(**criterion_params)
model = PointNetwork(params, criterion)
data_module._batch_size = 512

In [12]:
trainer = pl.Trainer(logger=TensorBoardLogger("lightning_logs"), max_epochs=10, gpus=1)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [13]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type                        | Params
-----------------------------------------------------------
0 | _criterion | RGBandModelReprojectionLoss | 0     
1 | backend    | Sequential                  | 51.6 K
-----------------------------------------------------------
51.6 K    Trainable params
0         Non-trainable params
51.6 K    Total params
0.206     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:610: LightningDeprecationWarning:

Relying on `self.log('val_loss', ...)` to set the ModelCheckpoint monitor is deprecated in v1.2 and will be removed in v1.4. Please, create your own `mc = ModelCheckpoint(monitor='your_monitor')` and use it as `Trainer(callbacks=[mc])`.



...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/trainer.py:897: UserWarning:

Detected KeyboardInterrupt, attempting graceful shutdown...



In [14]:
trainer.test(model, data_module.test_dataloader())

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'median_position_error': 0.4493865511215748,
 'median_rotation_error': 15.194313677822773,
 'point_count': 2000.0,
 'reconstruction_median_position_error': 0.004407721311686942,
 'reconstruction_median_rotation_error': 0.17272340737539726,
 'test_distance_loss': 0.2875872850418091,
 'test_good_point_ratio': 0.3717662990093231,
 'test_loss': 144.12823486328125,
 'test_reprojection_loss': 115.21223449707031}
--------------------------------------------------------------------------------


[{'test_loss': 144.12823486328125,
  'test_reprojection_loss': 115.21223449707031,
  'test_distance_loss': 0.2875872850418091,
  'test_good_point_ratio': 0.3717662990093231,
  'median_position_error': 0.4493865511215748,
  'median_rotation_error': 15.194313677822773,
  'reconstruction_median_position_error': 0.004407721311686942,
  'reconstruction_median_rotation_error': 0.17272340737539726,
  'point_count': 2000.0}]

In [16]:
task.close()